In [1]:
import os
os.chdir('..')
os.getcwd()

'/home/abraham-pc/Documents/paid_gigs/malaria_prediction_cv'

In [44]:
from sklearn.metrics import accuracy_score, classification_report, f1_score

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Activation, BatchNormalization, MaxPooling2D, GlobalAveragePooling2D, Dense, Multiply, Reshape, Attention, Flatten
import tensorflow_addons as tfa
from keras_self_attention import SeqSelfAttention 
import attention as attn
import mlflow


2024-01-13 12:09:23.214882: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-13 12:09:23.237696: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-13 12:09:23.237722: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-13 12:09:23.238401: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-13 12:09:23.242410: I tensorflow/core/platform/cpu_feature_guar

In [4]:
from src.features.build_features import train_generator, test_generator, val_generator, test_df

Found 144 validated image filenames belonging to 4 classes.
Found 36 validated image filenames belonging to 4 classes.
Found 46 validated image filenames.


In [5]:
def configure_gpu():

    try:
        gpus = tf.config.list_physical_devices('GPU')
        print(f"Found {len(gpus)} GPU(s): {gpus}")
    except Exception as e:
        print("Error configuring GPU: \n", e)
        print("Training will run on CPU")

    if gpus: # type: ignore
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("Memory growth enabled for all Physical GPUs")

configure_gpu()

Found 1 GPU(s): [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Memory growth enabled for all Physical GPUs


2024-01-13 12:09:24.322743: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-13 12:09:24.348905: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-13 12:09:24.349081: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [6]:
# Define the model

def build_dilated_cnn_with_attention(input_shape, num_classes, dilation_rate, attention_units):
    input_tensor = Input(shape=input_shape)

    # Dilated CNN block
    x = Conv2D(64, (3, 3), dilation_rate=dilation_rate, padding='same')(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = Conv2D(128, (3, 3), dilation_rate=dilation_rate, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = MaxPooling2D((2, 2))(x)

    # Attention mechanism
    # attention = GlobalAveragePooling2D()(x)
    attention = Attention()([x, x])
    attention = Dense(attention_units, activation='relu')(attention)
    attention = Dense(attention_units, activation='sigmoid')(attention)
    attention = Multiply()([x, attention])

    # Global average pooling and final dense layer
    x = GlobalAveragePooling2D()(attention)
    x = Dense(256, activation='relu')(x)
    output_tensor = Dense(num_classes, activation='softmax')(x)

    # Create the model
    model = Model(inputs=input_tensor, outputs=output_tensor)

    return model





In [7]:
# Parameters
input_shape = (224, 224, 3)
num_classes = 4
num_epochs = 100
batch_size = 32
patience = int(np.sqrt(num_epochs))
val_split = 0.1
dilation_rate = (2, 2)
attention_units = 128

In [8]:
# Build the model
model = build_dilated_cnn_with_attention(input_shape, num_classes, dilation_rate, attention_units)

# Compile the model and print a summary
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', tfa.metrics.F1Score(average='macro', num_classes=4)])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 224, 224, 64)         1792      ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 224, 224, 64)         256       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 224, 224, 64)         0         ['batch_normalization[0][0

2024-01-13 12:09:24.376677: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-13 12:09:24.376876: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-13 12:09:24.376996: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [47]:
def testdata_evaluation(model, train_generator, test_generator, test_df):
    mapping = train_generator.class_indices
    actual = test_df['species'].map(mapping)
    preds = model.predict(test_generator)
    predictions = np.argmax(preds, axis=1)
    accuracy = accuracy_score(actual, predictions)
    f1_weighted = f1_score(actual, predictions, average='weighted')
    return float(accuracy), float(f1_weighted)

In [48]:
# Define the MLflow experiment
experiment_name = "DCA_self_attention"
experiment = mlflow.set_experiment(experiment_name)

# Define paths
checkpoint_dir = './models/checkpoints/'
model_file = f'./models/model_{experiment_name+experiment.experiment_id}.keras'
os.makedirs( checkpoint_dir + experiment_name, exist_ok=True)
checkpoint_file = checkpoint_dir + experiment_name

# define callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_f1_score', 
    patience=patience, 
    mode='max',
    restore_best_weights=True
)

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_file, 
    monitor="val_f1_score",
    mode="max", 
    save_best_only=True,
    verbose=0
)

# Train the model
with mlflow.start_run(experiment_id=experiment.experiment_id):
    mlflow.log_params({
        "dilation_rate": dilation_rate,
        "attention_units": attention_units,
        "num_classes": num_classes,
        "input_shape": input_shape,
        # "batch_size": batch_size,
        "patience": patience,
        "num_epochs": num_epochs,
        "val_split": val_split
    })

    model.fit(
        train_generator,
        batch_size=batch_size,
        epochs=num_epochs,
        validation_data=val_generator,
        callbacks=[
            early_stopping,
            checkpoint
        ]
    )
    model.save(model_file)
    mlflow.tensorflow.autolog()
    accuracy, f1_weighted = testdata_evaluation(model, train_generator, test_generator, test_df)
    mlflow.log_metrics({
        "test accuracy": accuracy,
        "test f1_weighted": f1_weighted
    })
    mlflow.log_artifact(model_file)


2024/01/13 12:41:50 WARNING mlflow.tensorflow: Unrecognized dataset type <class 'keras.src.preprocessing.image.DataFrameIterator'>. Dataset logging skipped.
2024/01/13 12:41:50 WARNING mlflow.tensorflow: Unrecognized dataset type <class 'keras.src.preprocessing.image.DataFrameIterator'>. Dataset logging skipped.


Epoch 1/100
9/9 [==============================] - ETA: 0s - loss: 0.1495 - accuracy: 0.9583 - f1_score: 0.9586INFO:tensorflow:Assets written to: ./models/checkpoints/DCA_self_attention/assets


INFO:tensorflow:Assets written to: ./models/checkpoints/DCA_self_attention/assets


9/9 [==============================] - 8s 852ms/step - loss: 0.1495 - accuracy: 0.9583 - f1_score: 0.9586 - val_loss: 0.6052 - val_accuracy: 0.7222 - val_f1_score: 0.7267
Epoch 2/100
9/9 [==============================] - 6s 713ms/step - loss: 0.1104 - accuracy: 0.9792 - f1_score: 0.9760 - val_loss: 1.8400 - val_accuracy: 0.5833 - val_f1_score: 0.5856
Epoch 3/100
9/9 [==============================] - 6s 723ms/step - loss: 0.0907 - accuracy: 0.9722 - f1_score: 0.9666 - val_loss: 0.8212 - val_accuracy: 0.7222 - val_f1_score: 0.6909
Epoch 4/100
9/9 [==============================] - 6s 698ms/step - loss: 0.0960 - accuracy: 0.9792 - f1_score: 0.9767 - val_loss: 1.1036 - val_accuracy: 0.6667 - val_f1_score: 0.6437
Epoch 5/100
9/9 [==============================] - 6s 726ms/step - loss: 0.1763 - accuracy: 0.9375 - f1_score: 0.9356 - val_loss: 0.8028 - val_accuracy: 0.6111 - val_f1_score: 0.4895
Epoch 6/100
9/9 [==============================] - ETA: 0s - loss: 0.1675 - accuracy: 0.9444 - f1

INFO:tensorflow:Assets written to: ./models/checkpoints/DCA_self_attention/assets


...

9/9 [==============================] - 7s 807ms/step - loss: 0.0945 - accuracy: 0.9722 - f1_score: 0.9768 - val_loss: 1.0364 - val_accuracy: 0.8889 - val_f1_score: 0.8943
Epoch 12/100
9/9 [==============================] - 6s 704ms/step - loss: 0.0866 - accuracy: 0.9792 - f1_score: 0.9789 - val_loss: 1.3429 - val_accuracy: 0.7778 - val_f1_score: 0.7119
Epoch 13/100
9/9 [==============================] - 7s 716ms/step - loss: 0.0908 - accuracy: 0.9722 - f1_score: 0.9656 - val_loss: 0.9889 - val_accuracy: 0.8889 - val_f1_score: 0.8943
Epoch 14/100
9/9 [==============================] - 6s 713ms/step - loss: 0.1241 - accuracy: 0.9306 - f1_score: 0.9246 - val_loss: 1.3337 - val_accuracy: 0.6389 - val_f1_score: 0.5268
Epoch 15/100
9/9 [==============================] - 6s 712ms/step - loss: 0.1016 - accuracy: 0.9514 - f1_score: 0.9527 - val_loss: 1.1834 - val_accuracy: 0.7778 - val_f1_score: 0.7675
Epoch 16/100
9/9 [==============================] - 6s 719ms/step - loss: 0.0747 - accuracy: 

2024/01/13 12:44:08 WARNING mlflow.tensorflow: Failed to infer model signature: could not sample data to infer model signature: '>=' not supported between instances of 'slice' and 'int'
2024/01/13 12:44:08 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /tmp/tmpatdjlx_0/model/data/model/assets


INFO:tensorflow:Assets written to: /tmp/tmpatdjlx_0/model/data/model/assets


3/3 [==============================] - 1s 379ms/step


In [49]:
# best_model = tf.keras.models.load_model(checkpoint_file)
preds = model.predict(test_generator)

3/3 [==============================] - 1s 423ms/step


In [50]:
preds

array([[7.10999739e-05, 7.84875304e-02, 2.12693983e-03, 9.19314444e-01],
       [1.84933364e-04, 2.13963971e-01, 7.05408603e-02, 7.15310276e-01],
       [1.76824717e-04, 9.96332884e-01, 4.56311900e-05, 3.44461040e-03],
       [7.79897800e-06, 1.22584903e-03, 9.72494599e-05, 9.98669147e-01],
       [3.02500161e-03, 1.57949686e-01, 7.16817379e-02, 7.67343581e-01],
       [5.91811299e-01, 3.91916126e-01, 3.53719294e-03, 1.27354069e-02],
       [8.88795435e-01, 1.04605623e-01, 3.38223414e-04, 6.26072474e-03],
       [1.67701964e-03, 5.51013218e-04, 9.76245463e-01, 2.15264913e-02],
       [1.89687952e-01, 3.15860622e-02, 1.99848831e-01, 5.78877151e-01],
       [1.28906313e-03, 9.90541637e-01, 7.65050587e-04, 7.40418257e-03],
       [9.98642385e-01, 1.34481699e-03, 3.50061612e-07, 1.24972094e-05],
       [1.56073794e-02, 9.75244999e-01, 2.14831925e-05, 9.12621710e-03],
       [9.98974204e-01, 1.01291016e-03, 3.13667016e-07, 1.25641809e-05],
       [2.55360719e-05, 9.64505613e-01, 1.00120060e

In [51]:
# Classification Report for each type of attack
def print_f1_scores(y_true, y_pred):
    report = classification_report(y_true, y_pred, output_dict=True)

    for class_label, metric in report.items():
        try:
            f1_score_float = metric['f1-score']
            print(f"Class {class_label}: F1-score = {round(f1_score_float, 3)}")
        except:
            pass

In [52]:
predictions = np.argmax(preds, axis=1)

In [53]:
predictions

array([3, 3, 1, 3, 3, 0, 0, 2, 3, 1, 0, 1, 0, 1, 1, 1, 2, 3, 3, 3, 1, 1,
       3, 1, 1, 3, 0, 3, 1, 2, 3, 3, 3, 1, 3, 1, 2, 3, 1, 0, 0, 3, 0, 1,
       0, 3])

In [54]:
mapping = train_generator.class_indices

In [55]:
actual = test_df['species'].map(mapping)

In [56]:
actual

31    3
27    2
28    3
22    0
12    3
51    0
22    3
6     1
7     0
1     2
11    2
14    3
41    3
50    3
1     1
6     3
10    0
51    3
13    1
46    0
38    0
14    0
34    1
42    3
7     1
8     2
43    1
49    1
16    1
28    0
7     2
35    1
5     1
39    1
6     2
74    3
62    3
18    0
56    0
42    0
58    3
35    3
3     1
20    0
3     3
46    1
Name: species, dtype: int64

In [57]:
accuracy = accuracy_score(actual, predictions)
accuracy

0.21739130434782608

In [58]:
mapping

{'Falciparum': 0, 'Malariae': 1, 'Ovale': 2, 'Vivax': 3}

In [59]:
print_f1_scores(actual, predictions)

Class 0: F1-score = 0.19
Class 1: F1-score = 0.286
Class 2: F1-score = 0.0
Class 3: F1-score = 0.242
Class macro avg: F1-score = 0.18
Class weighted avg: F1-score = 0.209


In [46]:
f1_score(actual, predictions, average='weighted')

0.16703839638622248